In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/fifa19/data.csv')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
columns=['ID', 'Name', 'Age','Nationality',
       'Overall', 'Club', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type','Height', 'Weight','Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']


In [ ]:
df_raw=df[columns]

In [ ]:
df_raw.head()

In [ ]:
df_raw.isnull().sum()

In [ ]:
df_raw.drop('Club', inplace=True, axis=1) 


In [ ]:
df_raw.dropna(inplace=True, axis = 0) 

In [ ]:
df_raw.isnull().sum()

In [ ]:
df_raw.head()

In [ ]:
df_raw.describe(include='all')

In [ ]:
df_raw.drop('Weak Foot', inplace=True, axis=1)

In [ ]:
def con_val(val):
    temp_t=val[1:-1]
    if(len(temp_t)==0):
        temp_t='0'
    return float(temp_t)

In [ ]:
df_value=pd.DataFrame()
df_value['Value']=df_raw['Value']
df_raw['Value']=df_value['Value'].apply(con_val)


In [ ]:

# df_raw['Wage']=df_raw.Wage.map(lambda x: x[1:-1] )
df_value=pd.DataFrame()
df_value['Wage']=df_raw['Wage']
df_raw['Wage']=df_value['Wage'].apply(con_val)

In [ ]:
def convert_height(val):
    index=val.index("'")
    foot=val[0:index]
    inch=val[index+1:]
    height=float(foot)*30.48+float(inch)*2.54
    return height

In [ ]:
df_value=pd.DataFrame()
df_value['Height']=df_raw['Height']
df_raw['Height']=df_value['Height'].apply(convert_height)

In [ ]:
def convert_weight(val):
    index=val.index("lbs")
    weight=val[0:index]
    return float(weight)

In [ ]:
df_value=pd.DataFrame()
df_value['Weight']=df_raw['Weight']
df_raw['Weight']=df_value['Weight'].apply(convert_weight)

In [ ]:
col=['Age','Value', 'Wage',
       'Skill Moves','Crossing',
       'Finishing', 'HeadingAccuracy', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']
x=df_raw[col]


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
variables = df_raw[col]
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
vif["Features"] = variables.columns
vif

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)
log_overall = np.log(df_raw['Overall'])

# Then we add it to our data frame
df_raw['Overall'] = log_overall
y_log=df_raw['Overall']


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_log, test_size=0.2, random_state=365)

In [ ]:
reg = LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
y_hat = reg.predict(x_train)

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 
plt.scatter(y_train, y_hat)
plt.xlabel('Targets (y_train)',size=18)
plt.ylabel('Predictions (y_hat)',size=18)
plt.show()

In [ ]:
sns.distplot(y_train - y_hat)
plt.title("Residuals PDF", size=18)

In [ ]:
reg.score(x_train,y_train)

In [ ]:
reg.intercept_

In [ ]:
reg.coef_

In [ ]:
y_hat_test = reg.predict(x_test)
plt.scatter(y_test, y_hat_test, alpha=0.2)
plt.xlabel('Targets (y_test)',size=18)
plt.ylabel('Predictions (y_hat_test)',size=18)
plt.show()